<a href="https://colab.research.google.com/github/bdi2357/FootballData/blob/main/PlayersPersonalData/scraping_transfermarkt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scraping Transfermarkt

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

#Funcrions
## Not Player Specific
### scrape_premier_league_clubs
### scrape_players
## Player Specific
### scrape_player_injuries(player_url)
### parse_goals_by_minute_table(player_url)
### parse_all_goals_table(player_profile_url)


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# URL of the page to scrape
url = "https://www.transfermarkt.us/statistik/saisontransfers"
def scrape_transfer_data(url):
  # Headers to mimic a real browser request
  headers = {
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
  }

  # Make a GET request to fetch the raw HTML content
  response = requests.get(url, headers=headers)
  response.raise_for_status()  # Check if the request was successful

  # Parse the content of the request with BeautifulSoup
  soup = BeautifulSoup(response.content, 'html.parser')

  # Find the table containing the transfer data
  table = soup.find('table', class_='items')

  # Extract table headers

  #['1', 'Jude BellinghamAttacking Midfield', 'Jude Bellingham', 'Attacking Midfield', '20', '€120.00m', 'Real MadridLaLiga', 'Real Madrid', 'LaLiga', '€103.00m']
  tar_columns = ['#', 'Player position', 'Player', 'Position', 'Age','Market value', 'Team League', 'Team', 'League', 'fee']
  # Extract table rows
  rows = table.find_all('tr')

  # Extract data from each row
  data = []
  cnt =0
  cnt2=0
  for row in rows[1:]:  # Skip the header row
      cols = row.find_all('td')
      cols = [col.get_text(strip=True) for col in cols if len(col.get_text(strip=True) ) ]
      if cols:
          #print("*"*50)
          #print(cols)
          cnt +=1
          if len(cols) >1:
            cnt2 +=1
            #print(cols)
            data.append(cols)


  # Create a DataFrame
  print("headers")
  print(headers)
  print("data")
  for d  in data[:20]:
      print(len(d))
      print(d)

  df = pd.DataFrame(data, columns=tar_columns)
  return df

# Display the DataFrame to the user
#import ace_tools as tools; tools.display_dataframe_to_user(name="Transfermarkt Transfers", dataframe=df)
#"https://www.transfermarkt.us/statistik/saisontransfers"
# Optionally, save the DataFrame to a CSV file
df_transfers = []
a = True
cnt = 1
while a and cnt < 30:
  try:
    url =  "https://www.transfermarkt.us/statistik/saisontransfers?page=%d"%cnt
    df = scrape_transfer_data(url)
    df_transfers.append(df)
    cnt+=1
  except Exception as e:
    print(e)
    a = False
df = pd.concat(df_transfers)
df.to_csv('transfermarkt_transfers.csv', index=False)
print(df.shape)
print(len(rows))
print(cnt)


headers
{'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}
data
10
['1', 'Jude BellinghamAttacking Midfield', 'Jude Bellingham', 'Attacking Midfield', '20', '€120.00m', 'Real MadridLaLiga', 'Real Madrid', 'LaLiga', '€103.00m']
10
['2', 'Declan RiceDefensive Midfield', 'Declan Rice', 'Defensive Midfield', '24', '€90.00m', 'ArsenalPremier League', 'Arsenal', 'Premier League', '€116.60m']
10
['3', 'Moisés CaicedoDefensive Midfield', 'Moisés Caicedo', 'Defensive Midfield', '21', '€75.00m', 'ChelseaPremier League', 'Chelsea', 'Premier League', '€116.00m']
10
['4', 'Harry KaneCentre-Forward', 'Harry Kane', 'Centre-Forward', '30', '€90.00m', 'Bayern MunichBundesliga', 'Bayern Munich', 'Bundesliga', '€95.00m']
10
['5', 'Randal Kolo MuaniCentre-Forward', 'Randal Kolo Muani', 'Centre-Forward', '24', '€80.00m', 'Paris SGLigue 1', 'Paris SG', 'Ligue 1', '€95.00m']
10
['6', 'Josko GvardiolLeft-Back', 'Josko Gvardiol'

In [11]:
print (df.shape)
print(len(df_transfers))
sum([x.shape[0] for x in df_transfers])

(725, 10)
29


725

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_premier_league_clubs():
    # URL to the Premier League page on Transfermarkt
    url = 'https://www.transfermarkt.us/premier-league/startseite/wettbewerb/GB1'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send GET request to the league page
    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {url}")
        return None

    # Parse the league page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all clubs
    clubs_table = soup.find('table', {'class': 'items'})
    clubs = []
    for row in clubs_table.find_all('tr', {'class': ['odd', 'even']}):
        club_cell = row.find('td', {'class': 'hauptlink no-border-links'}).find('a')
        club_name = club_cell.text.strip()
        club_link = 'https://www.transfermarkt.us' + club_cell['href']
        clubs.append({'name': club_name, 'link': club_link})

    return clubs

# Example usage
premier_league_clubs = scrape_premier_league_clubs()
cnt = 0
for club in premier_league_clubs:
    cnt += 1
    print(f"Club: {club['name']}, Link: {club['link']}")
print(cnt)

Club: Manchester City, Link: https://www.transfermarkt.us/manchester-city/startseite/verein/281/saison_id/2023
Club: Arsenal FC, Link: https://www.transfermarkt.us/fc-arsenal/startseite/verein/11/saison_id/2023
Club: Chelsea FC, Link: https://www.transfermarkt.us/fc-chelsea/startseite/verein/631/saison_id/2023
Club: Liverpool FC, Link: https://www.transfermarkt.us/fc-liverpool/startseite/verein/31/saison_id/2023
Club: Tottenham Hotspur, Link: https://www.transfermarkt.us/tottenham-hotspur/startseite/verein/148/saison_id/2023
Club: Manchester United, Link: https://www.transfermarkt.us/manchester-united/startseite/verein/985/saison_id/2023
Club: Aston Villa, Link: https://www.transfermarkt.us/aston-villa/startseite/verein/405/saison_id/2023
Club: Newcastle United, Link: https://www.transfermarkt.us/newcastle-united/startseite/verein/762/saison_id/2023
Club: Brighton & Hove Albion, Link: https://www.transfermarkt.us/brighton-amp-hove-albion/startseite/verein/1237/saison_id/2023
Club: Crys

In [ ]:

import requests
import pandas as pd
from bs4 import BeautifulSoup

# Define the URL of the page to scrape
url = 'https://www.transfermarkt.us/manchester-city/startseite/verein/281/saison_id/2023'

# Set up headers to mimic a browser visit
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# Fetch the page content
response = requests.get(url, headers=headers)

# Check if the request was successful
if response.status_code != 200:
    print(f"Failed to retrieve data from {url}")
else:
    # Parse the page content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the squad table
    table = soup.find('table', {'class': 'items'})

    # Extract the table headers
    headers = []
    for th in table.find_all('th'):
        print(th.text)
        headers.append(th.text.strip())

    # Extract the table rows
    rows = []
    for row in table.find_all('tr', {'class': ['odd', 'even']}):
        cols = row.find_all('td')
        cols = [col.text.strip() for col in cols if len(col.text.strip())>0]
        rows.append(cols)

    # Create a DataFrame from the extracted data
    print(cols)
    #print(rows)
    print(headers)
    headers = ['#','Position', 'Player', 'Date of birth/Age', 'Nat.', 'Market value']
    df = pd.DataFrame(rows, columns=headers)

    # Display the DataFrame
    print("Manchester City Squad DataFrame:")
    print(df)
    df.to_csv('manchester_city_squad.csv', index=False)


#
Player
Date of birth/Age
Nat.
Market value
['19', 'Julián Álvarez            \n\n\n\n\n            Centre-Forward', 'Julián Álvarez', 'Centre-Forward', 'Jan 31, 2000 (24)', '€90.00m']
['#', 'Player', 'Date of birth/Age', 'Nat.', 'Market value']
Manchester City Squad DataFrame:
     #                                           Position  \
0   31          Ederson  \n\n\n\n\n            Goalkeeper   
1   18  Stefan Ortega            \n\n\n\n\n           ...   
2   33  Scott Carson            \n\n\n\n\n            ...   
3    3  Rúben Dias            \n\n\n\n\n            Ce...   
4   25  Manuel Akanji            \n\n\n\n\n           ...   
5    6  Nathan Aké            \n\n\n\n\n            Ce...   
6    5  John Stones            \n\n\n\n\n            C...   
7    -  Taylor Harwood-Bellis            \n\n\n\n\n   ...   
8   24  Josko Gvardiol            \n\n\n\n\n          ...   
9   21  Sergio Gómez            \n\n\n\n\n            ...   
10  97  Josh Wilson-Esbrand            \n\n\n\n\n

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_players(team_transfermarkt_url):
    # URL to the Manchester City page on Transfermarkt for the 2023 season
    #url = 'https://www.transfermarkt.us/manchester-city/startseite/verein/281/saison_id/2023'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send GET request to the team page
    response = requests.get(team_transfermarkt_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {team_transfermarkt_url}")
        return None

    # Parse the team page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all players
    players_table = soup.find('table', {'class': 'items'})
    players = []
    for row in players_table.find_all('tr', {'class': ['odd', 'even']}):
        player_cell = row.find('td', {'class': 'hauptlink'}).find('a')
        if player_cell != None:
          print("HERE")
          print(player_cell)
        player_name = player_cell.text.strip()
        player_link = 'https://www.transfermarkt.us' + player_cell['href']
        players.append({'name': player_name, 'link': player_link})


    return players

# Example usage
#team_transfermarkt_url = 'https://www.transfermarkt.us/manchester-city/startseite/verein/281/saison_id/2023'
team_transfermarkt_url = "https://www.transfermarkt.com/real-madrid/startseite/verein/418"
players = scrape_players(team_transfermarkt_url)
cnt = 0
for player in players:
    cnt += 1
    print(f"Player: {player['name']}, Link: {player['link']}")
print(cnt)

HERE
<a href="/thibaut-courtois/profil/spieler/108390">
                Thibaut Courtois            </a>
HERE
<a href="/andriy-lunin/profil/spieler/404839">
                Andriy Lunin            </a>
HERE
<a href="/kepa-arrizabalaga/profil/spieler/192279">
                Kepa Arrizabalaga            </a>
HERE
<a href="/eder-militao/profil/spieler/401530">
                Éder Militão            </a>
HERE
<a href="/antonio-rudiger/profil/spieler/86202">
                Antonio Rüdiger            </a>
HERE
<a href="/david-alaba/profil/spieler/59016">
                David Alaba<span class="verletzt-table icons_sprite" title="Cruciate ligament tear - Return unknown"> </span> </a>
HERE
<a href="/nacho-fernandez/profil/spieler/58884">
                Nacho Fernández<span class="kapitaenicon-table icons_sprite" title="Team captain"> </span> </a>
HERE
<a href="/ferland-mendy/profil/spieler/291417">
                Ferland Mendy            </a>
HERE
<a href="/fran-garcia/profil/spieler/3412

In [ ]:
players

[{'name': 'Thibaut Courtois',
  'link': 'https://www.transfermarkt.us/thibaut-courtois/profil/spieler/108390'},
 {'name': 'Andriy Lunin',
  'link': 'https://www.transfermarkt.us/andriy-lunin/profil/spieler/404839'},
 {'name': 'Kepa Arrizabalaga',
  'link': 'https://www.transfermarkt.us/kepa-arrizabalaga/profil/spieler/192279'},
 {'name': 'Éder Militão',
  'link': 'https://www.transfermarkt.us/eder-militao/profil/spieler/401530'},
 {'name': 'Antonio Rüdiger',
  'link': 'https://www.transfermarkt.us/antonio-rudiger/profil/spieler/86202'},
 {'name': 'David Alaba',
  'link': 'https://www.transfermarkt.us/david-alaba/profil/spieler/59016'},
 {'name': 'Nacho Fernández',
  'link': 'https://www.transfermarkt.us/nacho-fernandez/profil/spieler/58884'},
 {'name': 'Ferland Mendy',
  'link': 'https://www.transfermarkt.us/ferland-mendy/profil/spieler/291417'},
 {'name': 'Fran García',
  'link': 'https://www.transfermarkt.us/fran-garcia/profil/spieler/341264'},
 {'name': 'Daniel Carvajal',
  'link': 

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_player_injuries(player_url):
    # Extract the player ID from the URL
    player_id = player_url.split('/')[-1]

    # Construct the injuries URL
    injuries_url = f'https://www.transfermarkt.us/spieler/verletzungen/spieler/{player_id}'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send GET request to the injuries page
    response = requests.get(injuries_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {injuries_url}")
        return None

    # Parse the injuries page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the injuries table
    injuries_table = soup.find('table', {'class': 'items'})
    if not injuries_table:
        print(f"No injuries table found on the page {injuries_url}")
        return None

    injuries = []
    for row in injuries_table.find_all('tr', {'class': ['odd', 'even']}):
        cells = row.find_all('td')
        if len(cells) > 5:
            injury = {
                'season': cells[0].text.strip(),
                'injury': cells[1].text.strip(),
                'from': cells[2].text.strip(),
                'until': cells[3].text.strip(),
                'days': cells[4].text.strip(),
                'missed_games': cells[5].text.strip()
            }
            injuries.append(injury)

    return injuries

# Example usage
player_url = 'https://www.transfermarkt.us/joselu/profil/spieler/81999'#"https://www.transfermarkt.us/erling-haaland/profil/spieler/418560" #'https://www.transfermarkt.us/manuel-akanji/profil/spieler/284730'
player_injuries = scrape_player_injuries(player_url)

if player_injuries:
    for injury in player_injuries:
        print(injury)
pd.DataFrame(player_injuries)

{'season': '23/24', 'injury': 'stomach problems', 'from': 'Mar 16, 2024', 'until': 'Mar 21, 2024', 'days': '5 days', 'missed_games': '1'}
{'season': '23/24', 'injury': 'Ankle injury', 'from': 'Feb 23, 2024', 'until': 'Mar 1, 2024', 'days': '7 days', 'missed_games': '1'}
{'season': '22/23', 'injury': 'Muscle injury', 'from': 'Feb 7, 2023', 'until': 'Feb 24, 2023', 'days': '17 days', 'missed_games': '2'}
{'season': '16/17', 'injury': 'Achilles tendon problems', 'from': 'Apr 27, 2017', 'until': 'May 31, 2017', 'days': '34 days', 'missed_games': '4'}
{'season': '16/17', 'injury': 'Knee injury', 'from': 'Sep 12, 2016', 'until': 'Nov 28, 2016', 'days': '77 days', 'missed_games': '15'}
{'season': '14/15', 'injury': 'flu', 'from': 'Apr 3, 2015', 'until': 'Apr 6, 2015', 'days': '3 days', 'missed_games': '1'}
{'season': '14/15', 'injury': 'minor knock', 'from': 'Oct 8, 2014', 'until': 'Oct 17, 2014', 'days': '9 days', 'missed_games': '4'}
{'season': '13/14', 'injury': 'Muscle strain', 'from': 'A

,season,injury,from,until,days,missed_games
0,23/24,stomach problems,"Mar 16, 2024","Mar 21, 2024",5 days,1
1,23/24,Ankle injury,"Feb 23, 2024","Mar 1, 2024",7 days,1
2,22/23,Muscle injury,"Feb 7, 2023","Feb 24, 2023",17 days,2
3,16/17,Achilles tendon problems,"Apr 27, 2017","May 31, 2017",34 days,4
4,16/17,Knee injury,"Sep 12, 2016","Nov 28, 2016",77 days,15
5,14/15,flu,"Apr 3, 2015","Apr 6, 2015",3 days,1
6,14/15,minor knock,"Oct 8, 2014","Oct 17, 2014",9 days,4
7,13/14,Muscle strain,"Apr 26, 2014","May 3, 2014",7 days,2
8,09/10,Torn lateral ankle ligament,"Mar 18, 2010","Apr 7, 2010",20 days,3


In [ ]:
%%time
Injuries = {}
for player in players:
    player_url = player['link']
    player_injuries = scrape_player_injuries(player_url)
    if player_injuries:
        Injuries[player['name']] = player_injuries

No injuries table found on the page https://www.transfermarkt.us/spieler/verletzungen/spieler/948294
CPU times: user 3.35 s, sys: 9.51 ms, total: 3.36 s
Wall time: 24 s


In [ ]:
len(Injuries.items())

0

In [ ]:
Injuries.keys()

dict_keys(['Thibaut Courtois', 'Andriy Lunin', 'Kepa Arrizabalaga', 'Éder Militão', 'Antonio Rüdiger', 'David Alaba', 'Nacho Fernández', 'Ferland Mendy', 'Fran García', 'Daniel Carvajal', 'Lucas Vázquez', 'Aurélien Tchouaméni', 'Federico Valverde', 'Eduardo Camavinga', 'Toni Kroos', 'Luka Modric', 'Dani Ceballos', 'Jude Bellingham', 'Brahim Díaz', 'Arda Güler', 'Vinicius Junior', 'Rodrygo', 'Joselu'])

In [ ]:
pd.DataFrame(Injuries['Vinicius Junior'])

,season,injury,from,until,days,missed_games
0,23/24,Torn thigh muscle,"Nov 18, 2023","Jan 1, 2024",44 days,8
1,23/24,Hamstring muscle injury,"Aug 26, 2023","Sep 26, 2023",31 days,6
2,22/23,Knee problems,"May 24, 2023","May 27, 2023",3 days,2
3,21/22,Corona virus,"Dec 29, 2021","Jan 7, 2022",9 days,2
4,20/21,Intestinal virus,"Dec 19, 2020","Dec 22, 2020",3 days,1
5,18/19,Torn lateral knee ligament,"Mar 6, 2019","May 3, 2019",58 days,11


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def parse_all_tables(player_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send GET request to the player's page
    response = requests.get(player_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {player_url}")
        return None

    # Parse the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')
    if not tables:
        print(f"No tables found on the page {player_url}")
        return None

    # Create a dictionary to store DataFrames
    tables_dict = {}

    # Loop through all found tables
    for index, table in enumerate(tables):
        # Convert the table to a DataFrame
        df = pd.read_html(str(table))[0]
        # Store the DataFrame in the dictionary
        tables_dict[f'Table_{index + 1}'] = df

    return tables_dict

# Example usage
player_url = 'https://www.transfermarkt.us/erling-haaland/torenachminute/spieler/418560/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'
parsed_tables = parse_all_tables(player_url)

if parsed_tables:
    for table_name, df in parsed_tables.items():
        print(f"\n{table_name}:\n", df)



Table_1:
                                0  \
0              Filter by season:   
1                Filter by club:   
2  League ranking / League type:   
3         Filter by competition:   
4            Filter by position:   
5               Filter by coach:   

                                                   1   2  
0  All seasons 23/24 22/23 21/22 20/21 19/20 18/1... NaN  
1  All clubs Manchester City Borussia Dortmund Re... NaN  
2  All types First Tier Second Tier Domestic Cup ... NaN  
3  All competitions Bundesliga UEFA Super Cup Wor... NaN  
4  All positions Centre-Forward Left Winger Secon... NaN  
5  All coaches Lucien Favre Gaute Larsen Pep Guar... NaN  

Table_2:
    Season Competition             Competition.1  \
0   23/24         NaN            Premier League   
1   23/24         NaN          Champions League   
2   23/24         NaN                    FA Cup   
3   22/23         NaN            Premier League   
4   22/23         NaN          Champions League   
5   22

In [ ]:
parsed_tables["Table_2"].columns

Index(['Season', 'Competition', 'Competition.1',
       'reihenfolge ASC, saison_id DESC, verein_id ASC', 'Club', '1 - 15',
       '16 - 30', '31 - 45', '45+', '46 - 60', '61 - 75', '76 - 90', '90+',
       'Extra time', 'Total'],
      dtype='object')

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def parse_all_tables(player_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send GET request to the player's page
    response = requests.get(player_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {player_url}")
        return None

    # Parse the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all tables on the page
    tables = soup.find_all('table')
    if not tables:
        print(f"No tables found on the page {player_url}")
        return None

    # Create a dictionary to store DataFrames
    tables_dict = {}

    # Loop through all found tables
    for index, table in enumerate(tables):
        # Convert the table to a DataFrame
        df = pd.read_html(str(table))[0]
        # Clean and adjust the DataFrame
        df = clean_dataframe(df, index)
        # Store the DataFrame in the dictionary
        tables_dict[f'Table_{index + 1}'] = df

    return tables_dict

def clean_dataframe(df, table_index):
    # Fix the column names based on the number of columns
    if table_index == 1:  # Adjusting Table 2 specifically
        expected_columns = ['Minute Range', 'Goals', 'Penalties', 'Free Kicks', 'Own Goals', 'Headers', 'Right Foot', 'Left Foot', 'Other', 'Extra Time', 'Sum of Goals']
        if len(df.columns) == len(expected_columns):
            df.columns = expected_columns
        else:
            df.columns = [f'Column_{i}' for i in range(len(df.columns))]

    # Convert columns to appropriate data types
    df = df.apply(pd.to_numeric, errors='ignore')

    return df

# Example usage
player_url = 'https://www.transfermarkt.us/erling-haaland/torenachminute/spieler/418560/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'
parsed_tables = parse_all_tables(player_url)

if parsed_tables:
    for table_name, df in parsed_tables.items():
        print(f"\n{table_name}:\n", df)



Table_1:
                                0  \
0              Filter by season:   
1                Filter by club:   
2  League ranking / League type:   
3         Filter by competition:   
4            Filter by position:   
5               Filter by coach:   

                                                   1   2  
0  All seasons 23/24 22/23 21/22 20/21 19/20 18/1... NaN  
1  All clubs Manchester City Borussia Dortmund Re... NaN  
2  All types First Tier Second Tier Domestic Cup ... NaN  
3  All competitions Bundesliga World Cup qualific... NaN  
4  All positions Centre-Forward Left Winger Secon... NaN  
5  All coaches Lucien Favre Gaute Larsen Pep Guar... NaN  

Table_2:
    Column_0 Column_1                  Column_2  Column_3 Column_4 Column_5  \
0     23/24      NaN            Premier League       NaN        3        4   
1     23/24      NaN          Champions League       NaN        -        1   
2     23/24      NaN                    FA Cup       NaN        1        1   


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def parse_goals_by_minute_table(player_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send GET request to the player's page
    response = requests.get(player_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {player_url}")
        return None

    # Parse the page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the goals by minute table
    goals_table = soup.find('div', {'id': 'yw1'}).find('table', {'class': 'items'})
    if not goals_table:
        print(f"No goals by minute table found on the page {player_url}")
        return None

    # Convert the table to a DataFrame
    df = pd.read_html(str(goals_table))[0]

    # Correctly name the columns
    correct_columns = ['Minute Range', 'Goals', 'Penalties', 'Free Kicks', 'Own Goals', 'Headers', 'Right Foot', 'Left Foot', 'Other', 'Extra Time', 'Total']
    if len(df.columns) == len(correct_columns):
        df.columns = correct_columns
    else:
        print("Column length mismatch, please verify the table structure.")

    # Convert columns to appropriate data types
    df = df.apply(pd.to_numeric, errors='ignore')

    return df

# Example usage
player_url = 'https://www.transfermarkt.us/erling-haaland/torenachminute/spieler/418560'
goals_by_minute_df = parse_goals_by_minute_table(player_url)

if goals_by_minute_df is not None:
    print(goals_by_minute_df)


Column length mismatch, please verify the table structure.
   Season Competition             Competition.1  \
0   23/24         NaN            Premier League   
1   23/24         NaN          Champions League   
2   23/24         NaN                    FA Cup   
3   22/23         NaN            Premier League   
4   22/23         NaN          Champions League   
5   22/23         NaN                    FA Cup   
6   22/23         NaN                   EFL Cup   
7   21/22         NaN                Bundesliga   
8   21/22         NaN                 DFB-Pokal   
9   21/22         NaN          Champions League   
10  20/21         NaN                Bundesliga   
11  20/21         NaN                 DFB-Pokal   
12  20/21         NaN          Champions League   
13  20/21         NaN              DFL-Supercup   
14  19/20         NaN                Bundesliga   
15  19/20         NaN          Champions League   
16  19/20         NaN                 DFB-Pokal   
17  19/20         NaN  

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def construct_goals_by_minute_url(player_profile_url):
    # Extract the player ID from the profile URL
    player_id = player_profile_url.split('/')[-1]

    # Construct the URL for the "Goals by Minute" page
    goals_by_minute_url = f'https://www.transfermarkt.us/spieler/torenachminute/spieler/{player_id}/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'
    return goals_by_minute_url

def parse_goals_by_minute_table(player_profile_url):
    goals_by_minute_url = construct_goals_by_minute_url(player_profile_url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send GET request to the goals by minute page
    response = requests.get(goals_by_minute_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {goals_by_minute_url}")
        return None

    # Parse the goals by minute page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the goals by minute table
    goals_table = soup.find('div', {'id': 'yw1'}).find('table', {'class': 'items'})
    if not goals_table:
        print(f"No goals by minute table found on the page {goals_by_minute_url}")
        return None

    # Convert the table to a DataFrame
    df = pd.read_html(str(goals_table))[0]

    # Correctly name the columns
    correct_columns = ['Minute Range', 'Goals', 'Penalties', 'Free Kicks', 'Own Goals', 'Headers', 'Right Foot', 'Left Foot', 'Other', 'Extra Time', 'Total']
    if len(df.columns) == len(correct_columns):
        df.columns = correct_columns
    else:
        print("Column length mismatch, please verify the table structure.")

    # Convert columns to appropriate data types
    df = df.apply(pd.to_numeric, errors='ignore')

    return df

# Example usage
player_profile_url = 'https://www.transfermarkt.us/erling-haaland/profil/spieler/418560'
goals_by_minute_df = parse_goals_by_minute_table(player_profile_url)

if goals_by_minute_df is not None:
    print(goals_by_minute_df.head())


Column length mismatch, please verify the table structure.
  Season Competition     Competition.1  \
0  23/24         NaN    Premier League   
1  23/24         NaN  Champions League   
2  23/24         NaN            FA Cup   
3  22/23         NaN    Premier League   
4  22/23         NaN  Champions League   

   reihenfolge ASC, saison_id DESC, verein_id ASC Club 1 - 15 16 - 30 31 - 45  \
0                                             NaN    3      4       2       1   
1                                             NaN    -      1       -       1   
2                                             NaN    1      1       1       -   
3                                             NaN    3      6       8       -   
4                                             NaN    1      3       1       1   

  45+ 46 - 60 61 - 75 76 - 90 90+  Extra time  Total  
0   5       6       3       3   -          27    NaN  
1   2       1       1       -   -           6    NaN  
2   2       -       -       -   -   

In [ ]:
cols = goals_by_minute_df.columns
thresh = len(goals_by_minute_df) * 0.1
goals_by_minute_df.dropna(axis=1, thresh=thresh, inplace=True)


In [ ]:
cols

In [ ]:
cols_by_minute = ['Season', 'Competition', '1 - 15',
       '16 - 30', '31 - 45', '45+', '46 - 60', '61 - 75', '76 - 90', '90+',
       'Extra time', 'Total']

In [ ]:
goals_by_minute_df.columns = cols_by_minute

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def construct_goals_by_minute_url(player_profile_url):
    # Extract the player ID from the profile URL
    player_id = player_profile_url.split('/')[-1]

    # Construct the URL for the "Goals by Minute" page
    goals_by_minute_url = f'https://www.transfermarkt.us/spieler/torenachminute/spieler/{player_id}/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1'
    return goals_by_minute_url

def fill_na_in_columns(df, fill_values):
    """
    Fill NaN values in specific columns with given values.

    Parameters:
    df (pandas.DataFrame): The input DataFrame.
    fill_values (dict): A dictionary where keys are column names and values are the values to fill NaNs with.

    Returns:
    pandas.DataFrame: The DataFrame with NaN values in the specified columns filled.
    """
    for column, value in fill_values.items():
        df[column] = df[column].fillna(value)
    return df

def parse_goals_by_minute_table(player_profile_url):
    goals_by_minute_url = construct_goals_by_minute_url(player_profile_url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send GET request to the goals by minute page
    response = requests.get(goals_by_minute_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {goals_by_minute_url}")
        return None

    # Parse the goals by minute page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the goals by minute table
    goals_table = soup.find('div', {'id': 'yw1'}).find('table', {'class': 'items'})
    if not goals_table:
        print(f"No goals by minute table found on the page {goals_by_minute_url}")
        return None

    # Convert the table to a DataFrame
    df = pd.read_html(str(goals_table))[0]

    # Correctly name the columns
    correct_columns = ['Minute Range', 'Goals', 'Penalties', 'Free Kicks', 'Own Goals', 'Headers', 'Right Foot', 'Left Foot', 'Other', 'Extra Time', 'Total']
    if len(df.columns) == len(correct_columns):
        df.columns = correct_columns
    else:
        print("Column length mismatch, please verify the table structure.")

    # Convert columns to appropriate data types
    df = df.apply(pd.to_numeric, errors='ignore')
    cols = df.columns
    df.dropna(subset=['Season'], inplace=True)
    #df = fill_na_in_columns(df, {'Total': 0})
    #print(df.head())
    thresh = 1 # 2 #goals_by_minute_df.shape[0] * 0.05
    df.dropna(axis=1, thresh=thresh, inplace=True)
    #print("HERE1")
    # should drop if Season is NaN and impute total Nan to 0
    #print(df.head())
    cols_by_minute = ['Season', 'Competition', '1 - 15',
       '16 - 30', '31 - 45', '45+', '46 - 60', '61 - 75', '76 - 90', '90+','Extra time',
        'Total']
    #df.dropna(subset=['Season'], inplace=True)
    #df = fill_na_in_columns(df, {'Total': 0})
    #print("HERE2")
    #df.fillna(0, inplace=True
    #print(df.head())
    #print(df.shape)
    #print(len(cols_by_minute))
    #print(df.columns)
    if not 'Extra time' in df.columns:
        df.insert(, 'New_Column', 0) #df['Extra time'] = 0

    df.columns = cols_by_minute


    return df

# Example usage
player_profile_url = "https://www.transfermarkt.us/rodri/profil/spieler/357565" #"https://www.transfermarkt.us/nathan-ake/profil/spieler/177476" #"https://www.transfermarkt.us/manuel-akanji/profil/spieler/284730"#"https://www.transfermarkt.us/stefan-ortega/profil/spieler/85941"  #"https://www.transfermarkt.us/stefan-ortega/profil/spieler/85941" #'https://www.transfermarkt.us/erling-haaland/profil/spieler/418560'
goals_by_minute_df = parse_goals_by_minute_table(player_profile_url)

if goals_by_minute_df is not None:
    print(goals_by_minute_df.head())

player_profile_url = players[3]['link'] #" https://www.transfermarkt.us/eder-militao/profil/spieler/401530"
goals_by_minute_df = parse_goals_by_minute_table(player_profile_url)

if goals_by_minute_df is not None:
    print(goals_by_minute_df.head())


Column length mismatch, please verify the table structure.
  Season       Competition 1 - 15 16 - 30 31 - 45 45+ 46 - 60 61 - 75 76 - 90  \
0  23/24    Premier League      2       -       -   -       2       1       3   
1  23/24  Champions League      -       -       -   -       -       1       -   
2  22/23  Champions League      -       1       -   -       -       1       -   
3  22/23    Premier League      1       -       -   1       -       -       -   
4  21/22    Premier League      1       -       1   -       2       1       1   

  90+  Extra time  Total  
0   -           8      0  
1   -           1      0  
2   -           2      0  
3   -           2      0  
4   1           7      0  
Column length mismatch, please verify the table structure.
  Season       Competition 1 - 15 16 - 30 31 - 45 45+ 46 - 60 61 - 75 76 - 90  \
0  22/23            LaLiga      1       -       3   -       1       -       -   
1  22/23  Champions League      -       -       -   -       1       -  

In [ ]:
players

[{'name': 'Thibaut Courtois',
  'link': 'https://www.transfermarkt.us/thibaut-courtois/profil/spieler/108390'},
 {'name': 'Andriy Lunin',
  'link': 'https://www.transfermarkt.us/andriy-lunin/profil/spieler/404839'},
 {'name': 'Kepa Arrizabalaga',
  'link': 'https://www.transfermarkt.us/kepa-arrizabalaga/profil/spieler/192279'},
 {'name': 'Éder Militão',
  'link': 'https://www.transfermarkt.us/eder-militao/profil/spieler/401530'},
 {'name': 'Antonio Rüdiger',
  'link': 'https://www.transfermarkt.us/antonio-rudiger/profil/spieler/86202'},
 {'name': 'David Alaba',
  'link': 'https://www.transfermarkt.us/david-alaba/profil/spieler/59016'},
 {'name': 'Nacho Fernández',
  'link': 'https://www.transfermarkt.us/nacho-fernandez/profil/spieler/58884'},
 {'name': 'Ferland Mendy',
  'link': 'https://www.transfermarkt.us/ferland-mendy/profil/spieler/291417'},
 {'name': 'Fran García',
  'link': 'https://www.transfermarkt.us/fran-garcia/profil/spieler/341264'},
 {'name': 'Daniel Carvajal',
  'link': 

In [ ]:
%%time
goals_by_minute_d = {}
for player in players:
    try:
      player_url = player['link']
      goals_by_minute = parse_goals_by_minute_table(player_url)

      if goals_by_minute.shape[0] > 0 :
        if 'Season' in goals_by_minute.columns :

          goals_by_minute_d[player['name']] = goals_by_minute

    except Exception as e:
      print(e,player['name'],  player['link'] )

No goals by minute table found on the page https://www.transfermarkt.us/spieler/torenachminute/spieler/108390/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
'NoneType' object has no attribute 'shape' Thibaut Courtois https://www.transfermarkt.us/thibaut-courtois/profil/spieler/108390
No goals by minute table found on the page https://www.transfermarkt.us/spieler/torenachminute/spieler/404839/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
'NoneType' object has no attribute 'shape' Andriy Lunin https://www.transfermarkt.us/andriy-lunin/profil/spieler/404839
No goals by minute table found on the page https://www.transfermarkt.us/spieler/torenachminute/spieler/192279/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/plus/1
'NoneType' object has no attribute 'shape' Kepa Arrizabalaga https://www.transfermarkt.us/kepa-arrizabalaga/profil/spieler/192279
Column length mismatch, please verify the table structure.
Column length mismatch, please verify the table

In [ ]:
goals_by_minute_d.keys()

dict_keys(['Éder Militão', 'Antonio Rüdiger', 'David Alaba', 'Nacho Fernández', 'Ferland Mendy', 'Fran García', 'Daniel Carvajal', 'Lucas Vázquez', 'Aurélien Tchouaméni', 'Federico Valverde', 'Eduardo Camavinga', 'Toni Kroos', 'Luka Modric', 'Dani Ceballos', 'Jude Bellingham', 'Brahim Díaz', 'Arda Güler', 'Nico Paz', 'Vinicius Junior', 'Rodrygo', 'Joselu'])

In [ ]:
goals_by_minute_d['Vinicius Junior']

,Season,Competition,1 - 15,16 - 30,31 - 45,45+,46 - 60,61 - 75,76 - 90,90+,Extra time,Total
0,23/24,Champions League,-,2,-,-,1,1,2,-,-,6
1,23/24,LaLiga,2,3,1,1,1,6,1,-,-,15
2,23/24,Supercopa,2,-,1,-,-,-,-,-,-,3
3,22/23,Champions League,-,2,3,-,1,1,-,-,-,7
4,22/23,LaLiga,2,1,2,-,3,2,-,-,-,10
5,22/23,Copa del Rey,-,-,-,1,1,-,-,-,1,3
6,22/23,Club World Cup,1,-,1,-,-,1,-,-,-,3
7,21/22,Champions League,-,-,-,-,4,-,-,-,-,4
8,21/22,LaLiga,-,1,1,-,5,4,5,1,-,17
9,21/22,Supercopa,-,1,-,-,-,-,-,-,-,1


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def construct_all_goals_url(player_profile_url):
    # Extract the player ID from the profile URL
    player_id = player_profile_url.split('/')[-1]

    # Construct the URL for the "All Goals" page
    all_goals_url = f'https://www.transfermarkt.us/spieler/alletore/spieler/{player_id}/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/minute/0/torart/0/plus/1'
    return all_goals_url

def parse_all_goals_table(player_profile_url):
    all_goals_url = construct_all_goals_url(player_profile_url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send GET request to the all goals page
    response = requests.get(all_goals_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {all_goals_url}")
        return None

    # Parse the all goals page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Print all tables to debug
    tables = soup.find_all('table')
    """
    for i, table in enumerate(tables):
        print(f"Table {i}:")
        print(table.text[:500])  # Print the first 500 characters of the table to inspect its content
        print()
    """

    # Find the all goals table by looking for the specific header content
    goals_table = None
    for table in tables:
        if 'Competition' in table.text:  # Adjust this check based on the table's unique content
            goals_table = table
            break

    if not goals_table:
        print(f"No all goals table found on the page {all_goals_url}")
        return None

    # Convert the table to a DataFrame
    df = pd.read_html(str(goals_table))[0]

    # Drop columns where more than 90% of the values are NaNs
    thresh = len(df) * 0.1
    df.dropna(axis=1, thresh=thresh, inplace=True)

    # Convert columns to appropriate data types
    df = df.apply(pd.to_numeric, errors='ignore')

    return df

# Example usage
player_profile_url = "https://www.transfermarkt.us/kevin-de-bruyne/profil/spieler/88755" #'https://www.transfermarkt.us/scott-carson/profil/spieler/14555' #'https://www.transfermarkt.us/erling-haaland/profil/spieler/418560'
all_goals_df = parse_all_goals_table(player_profile_url)

if all_goals_df is not None:
    print(all_goals_df.shape)
    print(all_goals_df[-10:-1][[  'Date', 'Result' ,'Minute', 'Opponent.1','At score','Type of goal']])


(166, 12)
             Date        Result        Minute            Opponent.1  \
156           NaN           NaN           54'                   NaN   
157        5/9/23           1:1           67'           Real Madrid   
158  Season 23/24  Season 23/24  Season 23/24          Season 23/24   
159       1/13/24           2:3           74'        Newcastle (9.)   
160       2/13/24           1:3           10'         FC Copenhagen   
161        4/6/24           2:4           13'  Crystal Palace (14.)   
162           NaN           NaN           70'                   NaN   
163       4/17/24   4:5 on pens           76'           Real Madrid   
164       4/25/24           0:4           17'         Brighton (8.)   

         At score       Type of goal  
156           3:0  Right-footed shot  
157           1:1  Right-footed shot  
158  Season 23/24       Season 23/24  
159           2:2  Right-footed shot  
160           0:1  Right-footed shot  
161           1:1  Right-footed shot  
162   

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def construct_all_goals_url(player_profile_url):
    # Extract the player ID from the profile URL
    player_id = player_profile_url.split('/')[-1]

    # Construct the URL for the "All Goals" page
    all_goals_url = f'https://www.transfermarkt.us/spieler/alletore/spieler/{player_id}/saison//verein/0/liga/0/wettbewerb//pos/0/trainer_id/0/minute/0/torart/0/plus/1'
    return all_goals_url

def parse_all_goals_table(player_profile_url):
    all_goals_url = construct_all_goals_url(player_profile_url)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send GET request to the all goals page
    response = requests.get(all_goals_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {all_goals_url}")
        return None

    # Parse the all goals page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the all goals table by looking for the specific header content
    goals_table = None
    tables = soup.find_all('table')
    for table in tables:
        if 'Competition' in table.text:  # Check if this table contains the expected data
            goals_table = table
            break

    if not goals_table:
        print(f"No all goals table found on the page {all_goals_url}")
        return None

    # Convert the table to a DataFrame
    df = pd.read_html(str(goals_table))[0]

    # Drop rows that are header rows repeated in the data
    df = df[df['Competition'].str.contains("Season") == False]

    # Drop any rows that are completely NaN
    df.dropna(how='all', inplace=True)

    # Reset the index
    df.reset_index(drop=True, inplace=True)

    # Drop columns where more than 90% of the values are NaNs
    thresh = len(df) * 0.1
    df.dropna(axis=1, thresh=thresh, inplace=True)

    # Check the length of columns and reset the column names if the length matches
    expected_columns = ['Competition', 'Matchday', 'Date', 'Venue', 'For', 'Opponent', 'Result', 'Pos', 'Minute', 'At score', 'Type of goal', 'Goal assist']
    if len(df.columns) == len(expected_columns):
        df.columns = expected_columns
    else:
        print(f"Warning: Expected {len(expected_columns)} columns, but got {len(df.columns)} columns. Skipping renaming.")

    return df

# Example usage
player_profile_url = 'https://www.transfermarkt.us/erling-haaland/profil/spieler/418560'
all_goals_df = parse_all_goals_table(player_profile_url)

if all_goals_df is not None:
    print("All Goals DataFrame:")
    print(all_goals_df.columns)


All Goals DataFrame:
Index(['Competition', 'Competition.1', 'Matchday', 'Date', 'Venue', 'For',
       'For.1', 'Opponent', 'Opponent.1', 'Result', 'Pos.', 'Minute',
       'At score', 'Type of goal', 'Goal assist', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17'],
      dtype='object')


## Function that scrapes a list of Team Webpage Links

In [ ]:
#top_league(n) returns a list of links to the webpages of every club within the top n European leagues.
#i.e. Passing 1 as an argument will return links to the no.1 league in Europe- England. Passing 2 as an argument will return links of clubs in the 2 best leagues.
def top_league(n):

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    links = pageSoup.find_all("tr")

    site = 'https://www.transfermarkt.co.uk'
    s20_string = "/plus/?saison_id=2020"

    top_leagues = []
    for a in list(range(2,((n*2)+1),2)):
        top_leagues.append((site+(str(links[a]).split('href="',5)[2].split('"')[0])+s20_string))

    teams = []

    for a in top_leagues:

        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        links2 = pageSoup.find_all("a",{"class":"vereinprofil_tooltip"})
        site = 'https://www.transfermarkt.co.uk'
        s21_string = "/plus/?saison_id=2021"

        if a=="https://www.transfermarkt.co.uk/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2020" or a=="https://www.transfermarkt.co.uk/liga-portugal-bwin/startseite/wettbewerb/PO1/plus/?saison_id=2020" or a=="'https://www.transfermarkt.co.uk/eredivisie/startseite/wettbewerb/NL1/plus/?saison_id=2020'":
            for b in list(range(0,(18*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string)
        elif a== 'https://www.transfermarkt.co.uk/premier-liga/startseite/wettbewerb/RU1/plus/?saison_id=2020':
            for b in list(range(0,(16*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string)
        else:
            for b in list(range(0,(20*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string)
    return teams

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

def extract_market_values(player_market_value_url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    # Send GET request to the market value page
    response = requests.get(player_market_value_url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to retrieve data from {player_market_value_url}")
        return None

    # Parse the market value page
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find the JavaScript part containing the market values data
    scripts = soup.find_all('script')
    market_values_script = None
    for script in scripts:
        if 'Highcharts' in script.text:
            market_values_script = script.text
            break

    if not market_values_script:
        print(f"No market values data found on the page {player_market_value_url}")
        return None

    # Extract the market values data from the JavaScript
    market_values_data = re.search(r'\"data\":\[(.*?)\]', market_values_script)
    if not market_values_data:
        print(f"No market values data found in the JavaScript on the page {player_market_value_url}")
        return None

    # Process the extracted data
    market_values_str = market_values_data.group(1)
    market_values_list = market_values_str.split(',')
    market_values = [int(value) for value in market_values_list]

    # Extract the dates data from the JavaScript
    dates_data = re.search(r'categories\":\[(.*?)\]', market_values_script)
    if not dates_data:
        print(f"No dates data found in the JavaScript on the page {player_market_value_url}")
        return None

    # Process the extracted dates data
    dates_str = dates_data.group(1)
    dates_list = dates_str.split(',')
    dates = [date.strip('"') for date in dates_list]

    # Create a DataFrame from the market values and dates
    df = pd.DataFrame({'Date': dates, 'Market Value': market_values})

    return df

# Example usage
player_market_value_url = 'https://www.transfermarkt.us/kevin-de-bruyne/marktwertverlauf/spieler/88755'
market_values_df = extract_market_values(player_market_value_url)

if market_values_df is not None:
    print("Market Values DataFrame:")
    print(market_values_df)


## Function that scrapes a list of Individual Player Webpage Links

In [ ]:
#top_league_player(n) returns a list of links to the individual profile webpage for every player participating in Europe's top n number of leagues.
def top_league_player(n):

    headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

    page = "https://www.transfermarkt.co.uk/wettbewerbe/europa"
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    links = pageSoup.find_all("tr")

    site = 'https://www.transfermarkt.co.uk'
    s20_string = "/plus/?saison_id=2020"

    top_leagues = []
    for a in list(range(2,((n*2)+1),2)):
        top_leagues.append((site+(str(links[a]).split('href="',5)[2].split('"')[0])+s20_string))

    teams = []

    for a in top_leagues:

        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        links2 = pageSoup.find_all("a",{"class":"vereinprofil_tooltip"})
        site = 'https://www.transfermarkt.co.uk'
        s21_string = "/plus/?saison_id=2021"

        if a=="https://www.transfermarkt.co.uk/bundesliga/startseite/wettbewerb/L1/plus/?saison_id=2020" or a=="https://www.transfermarkt.co.uk/liga-portugal-bwin/startseite/wettbewerb/PO1/plus/?saison_id=2020" or a=="'https://www.transfermarkt.co.uk/eredivisie/startseite/wettbewerb/NL1/plus/?saison_id=2020'":
            for b in list(range(0,(18*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string)
        elif a== 'https://www.transfermarkt.co.uk/premier-liga/startseite/wettbewerb/RU1/plus/?saison_id=2020':
            for b in list(range(0,(16*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string)
        else:
            for b in list(range(0,(20*3),3)):
                teams.append((site+(str(links2[b]).split('href="')[1].split('">')[0])).split("/saison_id")[0]+s21_string)

    player_links = []
    for a in teams:
        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        site = 'https://www.transfermarkt.co.uk'

        links = pageSoup.find_all("a",{"class":"spielprofil_tooltip"})

        for b in list(range(0,len(links),2)):
            player_links.append(site+(str(links[b]).split('href="')[1].split('"')[0]))

    return player_links


## Scraping Player Contract Length Data

Contract dataframe: contract scrape from individual player pages. #Warning: IT MAY TAKE HOURS TO RUN but it WORKS!!!

In [ ]:
#iterating through every link in the list provided to it and then returns a dataframe of player data from the given webpages.
#top_league_player(5) returns a list of links to the individual profile webpage for every player participating in Europe's top 5 leagues.

site = 'https://www.transfermarkt.co.uk'
PlayerList = []
ContractList = []

headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

for a in top_league_player(5):

    page = a
    pageTree = requests.get(page, headers=headers)
    pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

    Player = pageSoup.find_all("meta",{"property":"og:title"})
    Contract = pageSoup.find_all("span",{"class":"dataValue"})

    PlayerList.append(str(Player).split('content="')[1].split(" - ")[0])
    try:
        ContractList.append((int(str(Contract[-1]).split(", ")[1].split("</span>")[0])-2021))
    except IndexError:
        ContractList.append("fail")

contract_df = pd.DataFrame({"Player":PlayerList,"Contract Years Left":ContractList})

In [ ]:
#Observing what our contract df looks like
contract_df

## Function that Scrapes and builds a dataframe of all Players' data

In [ ]:
#This function works when providing links of individual club pages.
#It iterates through every link in the list provided to it and then returns a dataframe of player data from the given webpages.
def build_df(links):

    PlayersList = []
    AgeList=[]
    NationList=[]
    ValuesList = []
    PositionsList=[]
    ClubList = []

    for a in links:

        headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}
        page = a
        pageTree = requests.get(page, headers=headers)
        pageSoup = BeautifulSoup(pageTree.content, 'html.parser')

        Players = pageSoup.find_all("img", {"class": "bilderrahmen-fixed lazy lazy"})
        Age = pageSoup.find_all("td", {"class": "zentriert"})
        Values = pageSoup.find_all("td", {"class": "rechts hauptlink"})
        Positions = pageSoup.find_all("td", {"class": ["zentriert rueckennummer bg_Torwart","zentriert rueckennummer bg_Abwehr","zentriert rueckennummer bg_Mittelfeld","zentriert rueckennummer bg_Sturm"]})
        Nationality = pageSoup.find_all("td", {"class": "zentriert"})
        Club = pageSoup.find_all("meta")

        for i in range(0,len(Values)):
            ValuesList.append(Values[i].text)

        for i in range(0,len(Players)):
            PlayersList.append(str(Players[i]).split('" class',1)[0].split('<img alt="',1)[1])

        for i in range(0,len(Positions)):
            PositionsList.append(str(Positions[i]).split('title="',1)[1].split('"><div')[0])

        for i in range(1,(len(Players)*3),3):
            AgeList.append(str(Age[i]).split("(",1)[1].split(")",1)[0])

        for i in range(2,(len(Players)*3),3):
            NationList.append(str(Nationality[i]).split('title="',1)[1].split('"/',1)[0])

        for i in range(0,len(Players)):
            ClubList.append(str(Club[6]).split('content="')[1].split(",")[0])


    #Initial uncleaned dataframe initiated
    df= pd.DataFrame({"Players":PlayersList,"Club":ClubList,"Position":PositionsList, "Age":AgeList, "Nationality":NationList,"Values":ValuesList,})

    #Missing Transfer Values were stored as '\xa0'. The line below replaces them as None values
    df.loc[(df.Values == '\xa0'),'Values']= None

    #Dataframe without missing transfer values
    df_droppednull = df.dropna()

    #Converting the Transfer Values data to float
    transfer_values = [a[1:len(a)-2] for a in df_droppednull['Values']]

    cleaned_values=[]
    for a in list(range(len(transfer_values))):
        if transfer_values[a].endswith('m'):
            cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-1])*1000000)
        elif transfer_values[a].endswith('Th.'):
            cleaned_values.append(float(transfer_values[a][:len(transfer_values[a])-3])*1000)
        else:
            cleaned_values.append(float(a))

    #Constructing the Final Dataframe
    final_df= pd.DataFrame({"Player":df_droppednull['Players'],"Club":df_droppednull['Club'],
                    "Age":df_droppednull['Age'],"Position":df_droppednull['Position'],
                    "Nation":df_droppednull['Nationality'],"Value":cleaned_values})

    #df ranked by transfer value
    ranked_df = final_df.sort_values('Value',ascending=False)

    return ranked_df

Building the dataframe of players from every Club Page

In [ ]:
#Building a dataframe by scraping information of the 98 teams from Europe's top 5 leagues - i.e. England, Spain, Italy, Germany & France
first_df = build_df(top_league(5))

### Seeing what our scraped dataset looks like

In [ ]:
#Seeing what our dataframe looks like
first_df

## Merging the contract length data onto the above dataframe

In [ ]:
#merging our dataframe with contract information
merged_df = pd.merge(first_df, contract_df, on='Player')

## Adding a League Column

In [ ]:
#Grouping Clubs by individual Leagues and making a separate "League" column.

England = ['Manchester City','Liverpool FC','Crystal Palace','Chelsea FC','Fulham FC','Brighton &amp; Hove Albion','West Bromwich Albion','Newcastle United','Leeds United','Aston Villa','Sheffield United','Manchester United','West Ham United','Leicester City','Tottenham Hotspur','Burnley FC','Southampton FC','Arsenal FC','Wolverhampton Wanderers','Everton FC']
Spain = ['SD Eibar','CA Osasuna','Athletic Bilbao','Real Betis Balompié','Valencia CF','Celta de Vigo','Levante UD','Cádiz CF','FC Barcelona','Villarreal CF','SD Huesca','Sevilla FC','Getafe CF','Deportivo Alavés','Real Valladolid CF','Granada CF','Real Madrid','Elche CF','Atlético de Madrid','Real Sociedad']
Italy = ['US Sassuolo','AC Milan','Juventus FC','Udinese Calcio','Genoa CFC','Cagliari Calcio','Inter Milan','FC Crotone','Hellas Verona','Benevento Calcio','Parma Calcio 1913','SSC Napoli','AS Roma','Bologna FC 1909','Torino FC','UC Sampdoria','SS Lazio','ACF Fiorentina','Atalanta BC','Spezia Calcio']
Germany = ['Bayern Munich','Hertha BSC','Borussia Dortmund','VfB Stuttgart','1.FSV Mainz 05','Arminia Bielefeld','Bayer 04 Leverkusen','SV Werder Bremen','1. FC Köln','FC Augsburg','VfL Wolfsburg','TSG 1899 Hoffenheim','RB Leipzig','SC Freiburg','1.FC Union Berlin','FC Schalke 04','Borussia Mönchengladbach','Eintracht Frankfurt']
France = ['AS Monaco','Montpellier HSC','FC Girondins Bordeaux','Paris Saint-Germain','Olympique Marseille','OGC Nice','Nîmes Olympique','FC Nantes','Stade Reims','FC Lorient','Stade Brestois 29','LOSC Lille','Stade Rennais FC','Dijon FCO','RC Strasbourg Alsace','RC Lens','AS Saint-Étienne','FC Metz','Olympique Lyon','SCO Angers']

LeagueList = []
for a in merged_df['Club']:
    if a in England:
        LeagueList.append('Premier League')
    elif a in Spain:
        LeagueList.append('La Liga')
    elif a in Italy:
        LeagueList.append('Serie A')
    elif a in Germany:
        LeagueList.append('Bundesliga')
    elif a in France:
        LeagueList.append('Ligue 1')
    else:
        LeagueList.append("UNKNOWN")

merged_df['League'] = LeagueList

In [ ]:
#looking at our dataframe with contract information appended.
merged_df

In [ ]:
#There are some duplicate rows
merged_df.drop_duplicates(keep="first",inplace=True)

In [ ]:
merged_df

## Exporting the Dataset

In [ ]:
# Install gitpath below if not already available on your environment.
#! pip install git+https://github.com/maxnoe/python-gitpath
import gitpath

#Getting the file path for the data file.
file_path = str(gitpath.root()) + '/data/transfermarkt_scrape_latest.xlsx'

In [ ]:
#Exporting data to .csv format in the repo's data folder.
merged_df.to_excel(file_path,index=False)